In [4]:
# coding: utf-8
from utils.clueweb_reader import ClueWebReader

In [2]:
%load_ext autoreload
%autoreload 2

In [71]:
reader = ClueWebReader('data/0013wb-88.warc.gz')
for i, (h, r) in enumerate(reader):
    print(h, r)

    if i > 4:
        break

('clueweb12-0013wb-88-00000', <warc.utils.FilePart instance at 0x106afdf80>)
('clueweb12-0013wb-88-00001', <warc.utils.FilePart instance at 0x106afddd0>)
('clueweb12-0013wb-88-00002', <warc.utils.FilePart instance at 0x106afdf38>)
('clueweb12-0013wb-88-00003', <warc.utils.FilePart instance at 0x106afddd0>)
('clueweb12-0013wb-88-00004', <warc.utils.FilePart instance at 0x106afdf80>)
('clueweb12-0013wb-88-00005', <warc.utils.FilePart instance at 0x106afddd0>)


In [69]:
r.header['WARC-TREC-ID']

'clueweb12-0013wb-88-00005'

# Record Counts (http://www.lemurproject.org/clueweb12/specs.php)
```
Segment	# Records
ClueWeb12_00	45,278,522
ClueWeb12_01	44,389,316
ClueWeb12_02	44,069,951
ClueWeb12_03	42,491,359
ClueWeb12_04	36,026,724
ClueWeb12_05	21,720,416
ClueWeb12_06	23,101,855
ClueWeb12_07	30,503,029
ClueWeb12_08	39,712,288
ClueWeb12_09	38,540,335
ClueWeb12_10	39,802,260
ClueWeb12_11	40,754,618
ClueWeb12_12	38,606,284
ClueWeb12_13	31,329,242
ClueWeb12_14	32,95,0768
ClueWeb12_15	37,716,513
ClueWeb12_16	34,996,028
ClueWeb12_17	34,051,249
ClueWeb12_18	40,074,978
ClueWeb12_19	36,903,637
Total	733,019,372
```